In [23]:
import json
import pandas as pd
from datetime import datetime

In [24]:
file_path = "../../data/raw/teslamotors_top_month_20250305.json"

with open(file_path, "r", encoding="utf-8") as file:
    json_data = json.load(file)
print(json_data)


[{'title': "Polestar's 'Trade In Your Tesla' $20,000 Deal Is Already A Hit", 'score': 1211, 'id': '1j1x782', 'url': 'https://insideevs.com/news/752184/polestar-3-tesla-lease-deal/', 'author': 'Generalaverage89', 'body': '', 'created_utc': '2025-03-02T23:54:53', 'num_comments': 324, 'permalink': '/r/teslamotors/comments/1j1x782/polestars_trade_in_your_tesla_20000_deal_is/', 'flair': 'General', 'nsfw': False, 'awards': 0}, {'title': 'Can confirm it looks better in real life', 'score': 906, 'id': '1iqczbr', 'url': 'https://i.redd.it/3p9lbronndje1.jpeg', 'author': 'wrongshirt', 'body': '', 'created_utc': '2025-02-16T04:00:51', 'num_comments': 182, 'permalink': '/r/teslamotors/comments/1iqczbr/can_confirm_it_looks_better_in_real_life/', 'flair': 'Vehicles - Model Y', 'nsfw': False, 'awards': 0}, {'title': 'New Model Y Ultra Red', 'score': 904, 'id': '1irnx4d', 'url': 'https://www.reddit.com/gallery/1irnx4d', 'author': 'Nikke_one', 'body': 'New Model Y Ultra Red photos from Tesla Meatpacking

In [25]:
df = pd.DataFrame(json_data)

In [26]:
df.head()

,title,score,id,url,author,body,created_utc,num_comments,permalink,flair,nsfw,awards
0,"Polestar's 'Trade In Your Tesla' $20,000 Deal ...",1211,1j1x782,https://insideevs.com/news/752184/polestar-3-t...,Generalaverage89,,2025-03-02T23:54:53,324,/r/teslamotors/comments/1j1x782/polestars_trad...,General,False,0
1,Can confirm it looks better in real life,906,1iqczbr,https://i.redd.it/3p9lbronndje1.jpeg,wrongshirt,,2025-02-16T04:00:51,182,/r/teslamotors/comments/1iqczbr/can_confirm_it...,Vehicles - Model Y,False,0
2,New Model Y Ultra Red,904,1irnx4d,https://www.reddit.com/gallery/1irnx4d,Nikke_one,New Model Y Ultra Red photos from Tesla Meatpa...,2025-02-17T22:17:04,149,/r/teslamotors/comments/1irnx4d/new_model_y_ul...,Vehicles - Model Y,False,0
3,Toronto Drops Tesla from Taxi Grant Program as...,789,1inxzez,https://driveteslacanada.ca/news/toronto-drops...,chrisdh79,,2025-02-13T00:16:58,183,/r/teslamotors/comments/1inxzez/toronto_drops_...,General,False,0
4,Tesla and Steak ‘n Shake partner to roll out 1...,783,1j09yvk,https://driveteslacanada.ca/news/tesla-and-ste...,chrisdh79,,2025-02-28T21:00:34,86,/r/teslamotors/comments/1j09yvk/tesla_and_stea...,Energy - Charging,False,0


In [27]:
# Feature engineering
df['created_utc'] = pd.to_datetime(df['created_utc'])
df['created_hour'] = df['created_utc'].dt.hour
df['created_dayofweek'] = df['created_utc'].dt.dayofweek

In [28]:
# Combine text fields (title + body) for NLP models
df['combined_text'] = df['title'].fillna('') + ' ' + df['body'].fillna('')

In [29]:
# Clean URLs and special chars from text
df['cleaned_text'] = df['combined_text'].str.replace(r'http\S+', '', regex=True).str.replace(r'\W+', ' ', regex=True)

In [30]:
# Encode categorical data
df['flair_encoded'] = pd.factorize(df['flair'])[0]


In [31]:
# Drop irrelevant columns
df = df.drop(columns=['url', 'id', 'permalink', 'created_utc', 'author'])

In [32]:
df.head()

,title,score,body,num_comments,flair,nsfw,awards,created_hour,created_dayofweek,combined_text,cleaned_text,flair_encoded
0,"Polestar's 'Trade In Your Tesla' $20,000 Deal ...",1211,,324,General,False,0,23,6,"Polestar's 'Trade In Your Tesla' $20,000 Deal ...",Polestar s Trade In Your Tesla 20 000 Deal Is ...,0
1,Can confirm it looks better in real life,906,,182,Vehicles - Model Y,False,0,4,6,Can confirm it looks better in real life,Can confirm it looks better in real life,1
2,New Model Y Ultra Red,904,New Model Y Ultra Red photos from Tesla Meatpa...,149,Vehicles - Model Y,False,0,22,0,New Model Y Ultra Red New Model Y Ultra Red ph...,New Model Y Ultra Red New Model Y Ultra Red ph...,1
3,Toronto Drops Tesla from Taxi Grant Program as...,789,,183,General,False,0,0,3,Toronto Drops Tesla from Taxi Grant Program as...,Toronto Drops Tesla from Taxi Grant Program as...,0
4,Tesla and Steak ‘n Shake partner to roll out 1...,783,,86,Energy - Charging,False,0,21,4,Tesla and Steak ‘n Shake partner to roll out 1...,Tesla and Steak n Shake partner to roll out 10...,2


In [33]:
import os

# Ensure the directory exists
save_dir = r"../../data/processed"

# Save to CSV
csv_path = os.path.join(save_dir, 'preprocessed_data.csv')
json_path = os.path.join(save_dir, 'preprocessed_data.json')
df.to_csv(csv_path, index=False)
# df.to_json(csv_path.replace('.csv', '.json'), orient='records', lines=True)
df.to_json(json_path, orient='records', indent=4) 
